In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from pandasql import sqldf
import requests as req
import pandas as pd
import sqlite3
import time
import os
import re

In [2]:
whole_page = ""

start_time = time.time()

for idx in range(5):
    url = f'https://store.steampowered.com/search/?specials=1&filter=topsellers&page={idx}'
    res = req.get(url)
    whole_page += res.text

time.time() - start_time

2.2505650520324707

In [3]:
start_time = time.time()
soup       = bs(whole_page, 'html.parser')
sales      = soup.select('div#search_resultsRows > a') 
time.time() - start_time

10.029686450958252

In [4]:
dbpath = 'steamsale.db'
conn   = sqlite3.connect(dbpath)
cursor = conn.cursor()

query = "DROP TABLE steamsale;"
cursor.execute(query)

In [11]:
query = """
CREATE TABLE IF NOT EXISTS steamsale(
id INTEGER PRIMARY KEY AUTOINCREMENT,
appid INTEGER NOT NULL,
percent INTEGER NOT NULL,
original INTEGER NOT NULL,
discounted INTEGER NOT NULL,
date DATE NOT NULL);"""

cursor.execute(query)

In [6]:
def preprop(string, dtype = 'discount'): 
    
    if dtype == 'discount': string = string.replace(' ', '').split('₩')[-1]
    string = string.replace(',', '').replace('₩', '')
    
    return int(string)
    

sale_list = []
for idx, sale in enumerate(sales):
    try:
        appid      = sale['data-ds-appid']

        percent    = sale.select('.search_discount > span')[0].text
        percent    = int(percent.replace('-', '').replace('%', ''))

        original   = preprop(sale.select('strike')[0].text)
        discounted = preprop(sale.select(".discounted")[0].text)

        sale_list.append([appid, title, percent, original, discounted, f'{Y}:{M}:{D}'])

        today = f'{Y}-{str(M).zfill(2)}-{str(D).zfill(2)}'
        query = f"""
                INSERT INTO steamsale
                (appid, percent, original, discounted, date)
                VALUES({appid}, {percent}, {original}, {discounted}, {today});
                """

        cursor.execute(query)
              
    except Exception as e: print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [ ]:
sorted(sale_list, key = lambda x: x[2], reverse = True)

In [12]:
query = "SELECT * FROM steamsale"
cursor.execute(query)

print(cursor.fetchall())

[(1, 289070, 90, 65000, 6500, 2012), (2, 648800, 33, 21000, 14070, 2012), (3, 381210, 65, 21000, 7350, 2012), (4, 227300, 75, 24800, 6200, 2012), (5, 368260, 33, 67800, 45430, 2012), (6, 397540, 85, 64900, 9730, 2012), (7, 477160, 70, 20500, 6150, 2012), (8, 1016920, 75, 19500, 4870, 2012), (9, 1286680, 50, 65000, 32500, 2012), (10, 270880, 75, 24800, 6200, 2012), (11, 495420, 50, 31000, 15500, 2012), (12, 1549180, 75, 11000, 2750, 2012), (13, 2251120, 15, 27000, 22950, 2012), (14, 1035760, 10, 17500, 15750, 2012), (15, 729000, 60, 20500, 8200, 2012), (16, 933820, 75, 33000, 8250, 2012), (17, 8870, 75, 32000, 8000, 2012), (18, 363890, 85, 86000, 12900, 2012), (19, 1599660, 33, 58800, 39400, 2012), (20, 645402, 84, 32500, 5200, 2012), (21, 947510, 88, 45000, 5400, 2012), (22, 960910, 50, 19900, 9950, 2012), (23, 559650, 75, 22000, 5500, 2012), (24, 1227890, 30, 20500, 14350, 2012), (25, 740080, 80, 14000, 2800, 2012), (26, 304212, 70, 22500, 6750, 2012), (27, 1209460, 30, 22500, 15750, 

In [8]:
with conn:
    with open('backup.sql', 'w') as f:
        for line in conn.iterdump():
            f.write('%s\n' % line)
        print('Completed')

Completed


In [10]:
query = f"""
            SELECT *
            FROM steamsale
            WHERE date={Y}-{str(M).zfill(2)}-{str(D).zfill(2)}
        """

cursor.execute(query)
cursor.fetchone()

(1, 289070, 90, 65000, 6500, 2012)